# New model types

## Build a model
Data scientists can still use the tools they are most comfortable with for the bulk of their workflow. Here, we can see using `torch` for a deep learning task of some random points.

In [2]:
import torch.nn as nn
import torch
import numpy as np

import vetiver


# Hyper-parameters
input_size = 1
output_size = 1
num_epochs = 60
learning_rate = 0.001

# # Toy dataset
x_train = np.array([[3.3], [4.4], [5.5], [6.71], [6.93], [4.168], 
                    [9.779], [6.182], [7.59], [2.167], [7.042], 
                    [10.791], [5.313], [7.997], [3.1]], dtype=np.float32)

y_train = np.array([[1.7], [2.76], [2.09], [3.19], [1.694], [1.573], 
                    [3.366], [2.596], [2.53], [1.221], [2.827], 
                    [3.465], [1.65], [2.904], [1.3]], dtype=np.float32)

# Linear regression model
model = nn.Linear(input_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
for epoch in range(num_epochs):
    # Convert numpy arrays to torch tensors
    inputs = torch.from_numpy(x_train)
    targets = torch.from_numpy(y_train)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

## Version a model

Just like we [saw with previous models](../superbowl-intro-py/notebook.ipynb) we can create a deployable model object from our `torch` model and version with pins.

In [3]:
v = vetiver.VetiverModel(
    model = model, 
    save_ptype = True, 
    ptype_data=x_train,
    model_name="torch", 
    description="A regression model for testing purposes"
)

In [5]:
import pins

board = pins.board_temp(allow_pickle_read=True)
vetiver.vetiver_pin_write(board, v)

Writing pin:
Name: 'torch'
Version: 20220809T165227Z-999a5


## Deploy a model locally
Next, we can make a local API endpoint with `VetiverAPI()` and start it with `.run()`

In [6]:
api = vetiver.VetiverAPI(v, check_ptype=True)
api.run()

## Custom models

Sometimes, you might use a model type that is not natively supported by Vetiver, or maybe you are writing a completely custom model that is not associated with any package at all. You are still able to deploy these models using a `VetiverHandler()`.